# Rutas almacen a c/polígono

In [ ]:
import cv2
import numpy as np
import networkx as nx
from math import sqrt

# ===== CONFIGURACIÓN =====
IMG_PATH = "Color.jpg"
VELOCIDAD_IDA = (2500 / 36)/60  # 25 km/h -> m/s
VELOCIDAD_REGRESO = (4000 / 36)/60  # 40 km/h -> m/s
ESCALA = 100 / 0.6  # 100m reales = 0.6 cm en imagen (px->m: px/ESCALA)

# ===== PROCESAMIENTO DE IMAGEN =====
img = cv2.imread(IMG_PATH)
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# 1. Detectar almacén (verde)
mask_verde = cv2.inRange(hsv, (40, 100, 100), (80, 255, 255))
contours_verde, _ = cv2.findContours(mask_verde, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
if not contours_verde:
    raise ValueError("No se detectó el almacén verde")

M = cv2.moments(contours_verde[0])
almacen = (int(M["m10"]/M["m00"]), int(M["m01"]/M["m00"]))

# 2. Detectar líneas amarillas (caminos)
mask_amarillo = cv2.inRange(hsv, (20, 100, 100), (40, 255, 255))
kernel = np.ones((3,3), np.uint8)
mask_amarillo = cv2.morphologyEx(mask_amarillo, cv2.MORPH_CLOSE, kernel)
contours_amarillo, _ = cv2.findContours(mask_amarillo, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# 3. Detectar polígonos (magenta)
mask_magenta = cv2.inRange(hsv, (140, 50, 50), (170, 255, 255))
contours_magenta, _ = cv2.findContours(mask_magenta, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# ===== CONSTRUIR GRAFO DE CAMINOS =====
G = nx.Graph()

# Añadir todas las aristas amarillas
for cnt in contours_amarillo:
    for i in range(len(cnt)-1):
        p1 = tuple(cnt[i][0])
        p2 = tuple(cnt[i+1][0])
        distancia = sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)
        G.add_edge(p1, p2, weight=distancia)

# Conectar almacén a la red amarilla más cercana
punto_conexion_almacen = None
min_dist = float('inf')
for node in G.nodes():
    dist = sqrt((almacen[0]-node[0])**2 + (almacen[1]-node[1])**2)
    if dist < min_dist and dist < 50:  # 50px máximo
        min_dist = dist
        punto_conexion_almacen = node

if punto_conexion_almacen:
    G.add_edge(almacen, punto_conexion_almacen, weight=min_dist)
else:
    raise ValueError("El almacén no está conectado a las rutas amarillas")

# ===== CALCULAR RUTAS A POLÍGONOS =====
resultados = []
img_rutas = img.copy()

for i, cnt in enumerate(contours_magenta):
    # Extraer vértices del polígono
    vertices = [tuple(p[0]) for p in cnt]
    
    # Encontrar la ruta más corta a cualquier vértice
    distancia_min = float('inf')
    ruta_optima = None
    
    for vertice in vertices:
        try:
            # Conectar vértice a la red amarilla más cercana
            punto_conexion = None
            min_dist_v = float('inf')
            for node in G.nodes():
                dist = sqrt((vertice[0]-node[0])**2 + (vertice[1]-node[1])**2)
                if dist < min_dist_v and dist < 50000:  # 50px máximo
                    min_dist_v = dist
                    punto_conexion = node
            
            if punto_conexion:
                G.add_edge(vertice, punto_conexion, weight=min_dist_v)
                path = nx.shortest_path(G, source=almacen, target=vertice, weight='weight')
                distancia = sum(G[u][v]['weight'] for u,v in zip(path[:-1], path[1:]))
                
                if distancia < distancia_min:
                    distancia_min = distancia
                    ruta_optima = path
        except:
            continue
    
    # Calcular tiempos si se encontró ruta
    if ruta_optima:
        distancia_metros = distancia_min / ESCALA
        tiempo_ida = distancia_metros / VELOCIDAD_IDA
        tiempo_regreso = distancia_metros / VELOCIDAD_REGRESO
        
        resultados.append({
            'Poligono': i+1,
            'Distancia (m)': round(distancia_metros, 2),
            'Tiempo ida (s)': round(tiempo_ida, 1),
            'Tiempo regreso (s)': round(tiempo_regreso, 1)
        })
        
        # Dibujar ruta
        for j in range(len(ruta_optima)-1):
            cv2.line(img_rutas, ruta_optima[j], ruta_optima[j+1], (0,0,255), 3)
        
        # Marcar polígono
        cv2.putText(img_rutas, str(i+1), vertices[0], cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
    else:
        resultados.append({
            'Poligono': i+1,
            'Distancia (m)': 'N/A',
            'Tiempo ida (s)': 'N/A',
            'Tiempo regreso (s)': 'N/A'
        })

# ===== GUARDAR RESULTADOS =====
# 1. Exportar CSV
import csv
with open('resultados_rutas.csv', 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=resultados[0].keys())
    writer.writeheader()
    writer.writerows(resultados)

# 2. Guardar imagen con rutas
cv2.imwrite('rutas_detectadas.jpg', img_rutas)

# 3. Mostrar resumen
print("╔══════════╦══════════════╦════════════╦════════════════╗")
print("║ Polígono ║ Distancia (m)║ Tiempo ida ║ Tiempo regreso ║")
print("╠══════════╬══════════════╬════════════╬════════════════╣")
for res in resultados:
    print(f"║{res['Poligono']:^10}║{str(res['Distancia (m)']):^14}║{str(res['Tiempo ida (s)']):^12}║{str(res['Tiempo regreso (s)']):^16}║")
print("╚══════════╩══════════════╩════════════╩════════════════╝")

# Rutas color

In [ ]:
import cv2
import numpy as np
import networkx as nx
from math import sqrt
import random
import csv

# ===== CONFIGURACIÓN =====
IMG_PATH = "Color.jpg"
VELOCIDAD_IDA = (2500 / 36)/60  # 25 km/h -> m/s
VELOCIDAD_REGRESO = (4000 / 36)/60  # 40 km/h -> m/s
ESCALA = 100 / 0.6  # 100m reales = 0.6 cm en imagen (px->m: px/ESCALA)

COLORES = [
    (255, 0, 0),    # Rojo
    (0, 255, 0),    # Verde
    (0, 0, 255),    # Azul
    (255, 255, 0),  # Cian
    (255, 0, 255),  # Magenta
    (0, 255, 255),  # Amarillo
    (128, 0, 128),  # Púrpura
    (0, 128, 128)   # Verde oscuro
]

# ===== PROCESAMIENTO DE IMAGEN =====
img = cv2.imread(IMG_PATH)
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
img_rutas = img.copy()

# 1. Detectar almacén (verde)
mask_verde = cv2.inRange(hsv, (40, 100, 100), (80, 255, 255))
contours_verde, _ = cv2.findContours(mask_verde, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
if not contours_verde:
    raise ValueError("No se detectó el almacén verde")

M = cv2.moments(contours_verde[0])
almacen = (int(M["m10"]/M["m00"]), int(M["m01"]/M["m00"]))
cv2.circle(img_rutas, almacen, 10, (0, 0, 255), -1)  # Destacar almacén

# 2. Detectar líneas amarillas (caminos)
mask_amarillo = cv2.inRange(hsv, (20, 100, 100), (40, 255, 255))
kernel = np.ones((3,3), np.uint8)
mask_amarillo = cv2.morphologyEx(mask_amarillo, cv2.MORPH_CLOSE, kernel)
contours_amarillo, _ = cv2.findContours(mask_amarillo, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# 3. Detectar polígonos (magenta)
mask_magenta = cv2.inRange(hsv, (140, 50, 50), (170, 255, 255))
contours_magenta, _ = cv2.findContours(mask_magenta, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# ===== CONSTRUIR GRAFO =====
G = nx.Graph()

# Añadir aristas amarillas
for cnt in contours_amarillo:
    for i in range(len(cnt)-1):
        p1 = tuple(cnt[i][0])
        p2 = tuple(cnt[i+1][0])
        distancia = sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)
        G.add_edge(p1, p2, weight=distancia)

# Conectar almacén a la red
punto_conexion_almacen = None
min_dist = float('inf')
for node in G.nodes():
    dist = sqrt((almacen[0]-node[0])**2 + (almacen[1]-node[1])**2)
    if dist < min_dist and dist < 50:
        min_dist = dist
        punto_conexion_almacen = node

if punto_conexion_almacen:
    G.add_edge(almacen, punto_conexion_almacen, weight=min_dist)
else:
    raise ValueError("Almacén no conectado a rutas")

# ===== CALCULAR RUTAS =====
resultados = []
for i, cnt in enumerate(contours_magenta):
    vertices = [tuple(p[0]) for p in cnt]
    color = COLORES[i % len(COLORES)]  # Asignar color único

    # Encontrar mejor ruta
    distancia_min = float('inf')
    ruta_optima = None

    for vertice in vertices:
        try:
            # Conectar vértice a red
            punto_conexion = None
            min_dist_v = float('inf')
            for node in G.nodes():
                dist = sqrt((vertice[0]-node[0])**2 + (vertice[1]-node[1])**2)
                if dist < min_dist_v and dist < 50:
                    min_dist_v = dist
                    punto_conexion = node

            if punto_conexion:
                G_temp = G.copy()
                G_temp.add_edge(vertice, punto_conexion, weight=min_dist_v)
                path = nx.shortest_path(G_temp, source=almacen, target=vertice, weight='weight')
                distancia = sum(G_temp[u][v]['weight'] for u,v in zip(path[:-1], path[1:]))

                if distancia < distancia_min:
                    distancia_min = distancia
                    ruta_optima = path
        except:
            continue

    # Procesar resultados
    if ruta_optima:
        distancia_metros = distancia_min / ESCALA
        tiempo_ida = distancia_metros / VELOCIDAD_IDA
        tiempo_regreso = distancia_metros / VELOCIDAD_REGRESO

        resultados.append({
            'Poligono': i+1,
            'Distancia (m)': round(distancia_metros, 2),
            'Tiempo ida (s)': round(tiempo_ida, 1),
            'Tiempo regreso (s)': round(tiempo_regreso, 1),
            'Color': color
        })

        # Dibujar ruta
        for j in range(len(ruta_optima)-1):
            cv2.line(img_rutas, ruta_optima[j], ruta_optima[j+1], color, 3)

        # Marcar polígono
        cv2.putText(img_rutas, str(i+1), vertices[0],
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
    else:
        resultados.append({
            'Poligono': i+1,
            'Distancia (m)': 'N/A',
            'Tiempo ida (s)': 'N/A',
            'Tiempo regreso (s)': 'N/A',
            'Color': (0,0,0)
        })

# ===== LEYENDA =====
# Get the width of the main image
img_width = img_rutas.shape[1]
legend_height = 150 # Keep the height the same
leyenda = np.zeros((legend_height, img_width, 3), dtype=np.uint8) + 255  # Fondo blanco con ancho de la imagen principal

cv2.putText(leyenda, "Leyenda:", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,0), 2)

for i, res in enumerate(resultados[:8]):  # Mostrar primeros 8
    if res['Color'] != (0,0,0):
        cv2.line(leyenda, (10, 50+i*20), (30, 50+i*20), res['Color'], 3)
        cv2.putText(leyenda, f"Poligono {res['Poligono']}", (40, 55+i*20),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1)

# Combinar imagen y leyenda
img_final = np.vstack([img_rutas, leyenda])

# ===== GUARDAR RESULTADOS =====
cv2.imwrite('rutas_color.jpg', img_final)

# Exportar CSV
with open('resultados_rutas.csv', 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=['Poligono', 'Distancia (m)', 'Tiempo ida (s)', 'Tiempo regreso (s)'])
    writer.writeheader()
    writer.writerows([{k:v for k,v in res.items() if k != 'Color'} for res in resultados])

print("Proceso completado. Ver 'rutas_color.jpg' y 'resultados_rutas.csv'")